In [193]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [194]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df.fillna(0)

df.sample(20)



,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
241597,0x5b1c9625ebc55cc415638070d923ac9bcf32981b,out,0x961ec1186cd7e4cf9d5b40fcd1ce23823d59b8d8,0,transfer,6.025325e+05,1,1,Ethereum
255573,0x20eadfcaf91bd98674ff8fc341d148e1731576a4,in,0x99a58482bd75cbab83b27ec03ca68ff489b5788f,0,exchange_multiple,5.975954e+04,1,1,Ethereum
1231,0xa0894a415c4f246ce95bae718849579c099cc1d2,out,OTHER,OTHER,OTHER,1.590134e+04,12,12,Arbitrum
16818,0x30204c1f6f43747a3053e08a959ac7c0a13d210e,in,0xeb5ed670e8a512a7ecd34188f8b5756dba23d1c7,0,sendMultiSigToken,4.305225e+05,1,1,Ethereum
140763,0xcd531ae9efcce479654c4926dec5f6209531ca7b,out,0xfe15dcef755bae961e7e8fc08deb1254e8ab1df9,0,transfer,6.983770e+05,2,2,Ethereum
37182,0xb8330d1829159b18d0e081a7080f00440bc4341c,in,0x7d63e3e9ffbd4994dc253415a364e7d9883ac16d,0,transfer,1.146626e+06,1,1,Ethereum
152449,0xc4d300019f587145cfd9b42875073b3ad4bb0296,in,0x06b099e3ba7a8d7c17d7eb10b7f4323a0ce053e1,0,transfer,2.415990e+04,1,1,Ethereum
163113,0x77134cbc06cb00b66f4c7e623d5fdbf6777635ec,in,0x869ad74a0c59881db89c7388343688325f8350fc,0,transfer,4.904008e+04,1,1,Ethereum
240514,0x6c16b5511718e396ede3d1073b2103b7de9b5d99,in,0x3b25b94067bfb7dcbf4e817b76db1dffd4258bb8,0,transfer,2.450599e+05,1,1,Ethereum
34480,0x29d5527caa78f1946a409fa6acaf14a0a4a0274b,out,0xfce289a89cef5bd9fa97c648b649679cc76e8221,0,transfer,1.448094e+04,1,1,Ethereum


In [195]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [196]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/4210915753.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [197]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net',\
                # ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share','address'
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [198]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [199]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_action.reset_index(inplace=True)
df_action.name = 'df_action'

display(df_action)

,chain,project,contract,method,usd_value,usd_value_in,usd_value_out,usd_value_net,address
0,Arbitrum,0x0000000000be6d8381e3a01c19e3b3c2b6d9c7cd,0,0xed5b0df3,5.829281e+06,695814.462200,5.133467e+06,695814.462200,1
1,Arbitrum,0x000000d4728ded3cc178ffa58355e6b57b931ee2,0,0xed5b0df3,9.211949e+06,318280.444237,8.893669e+06,318280.444237,1
2,Arbitrum,0x000c3ff63086a0a34b3a4750a66adf206753fc6b,0,transfer,1.007082e+04,10070.820000,0.000000e+00,10070.820000,1
3,Arbitrum,0x00680642adb56ea659d96a91e81099c43886f513,0,transfer,4.997175e+05,418897.352500,8.082018e+04,418897.352500,1
4,Arbitrum,0x008ba48b1c005edbc76d75c7143c868576f1fbbf,0,transfer,1.129990e+04,0.000000,1.129990e+04,0.000000,1
...,...,...,...,...,...,...,...,...,...
139229,Optimism,Wepiggy,PEther,borrow,2.318055e+05,231805.500000,0.000000e+00,231805.500000,1
139230,Optimism,Wepiggy,PEther,repayBorrow,1.699907e+04,0.000000,1.699907e+04,0.000000,1
139231,Optimism,Woofi,0,withdraw,1.004902e+05,100490.201009,0.000000e+00,100490.201009,1
139232,Optimism,Woofi,WooSuperChargerVault,deposit,1.004344e+05,0.000000,1.004344e+05,0.000000,1


In [200]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

# df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

In [201]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

        print(sumdf[['usd_value_out_pct_share_100']].sum())

Dollar-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
2562,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,9.523727,9.255335
2561,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,5.702290,4.862586
1927,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,5.504098,0.000000
2563,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,4.944629,5.567669
2015,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,3.998533,0.000000
536,Arbitrum,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3 - 0...,3.286356,1.529717
343,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,3.059880,0.000000
2568,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,2.852370,3.406012
2020,Arbitrum,gmx,gmx - Vault | 0x364e2311,2.278467,0.000000
1891,Arbitrum,OTHER,OTHER - OTHER | OTHER,2.271972,2.345757


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
663,Optimism,Celer,Celer - Bridge | addLiquidity,17.775668,0.000000
944,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,15.668466,15.976778
761,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,5.628581,0.000000
950,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,3.548631,3.762533
736,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,2.776817,4.930665
633,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,2.594831,0.006871
375,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,0xb1feccccaad6429899a9401dcb19f58a19d1afe6 - 0...,2.323820,0.000000
666,Optimism,Celer,Celer - Bridge | send,1.833812,0.000000
741,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,1.822989,0.000000
804,Optimism,Socket,Socket - 0 | outboundTransferTo,1.733445,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
576,Optimism,Uniswap,23.300819,25.730368
533,Optimism,Celer,20.354813,24.457077
553,Optimism,Multichain,6.270284,6.020605
549,Optimism,Kyberswap,5.180304,5.265309
524,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,2.639540,0.006871
526,Optimism,Aave,2.598473,4.172976
577,Optimism,Uniswap V3,2.470609,3.366635
571,Optimism,Sonne Finance,2.432443,2.811540
357,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,2.323820,0.000000
537,Optimism,Dforce,1.920007,1.210225


usd_value_out_pct_share_100    100.0
dtype: float64


In [202]:
print('Address-Weighted')
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # print(tdf)
        tdf = tdf[tdf['address'] >=20]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        print(sumdf.columns)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project','address']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )

Address-Weighted


KeyError: 'address'